In [1]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
import os
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
import nltk
import warnings
warnings.filterwarnings("ignore")

AUGMENT_WITH_NEUTRAL_ARGS = True
data_dir = "data/argumentation"

train_df = pd.read_csv(os.path.join(data_dir, 'train_iam.tsv'), sep='\t')
dev_df = pd.read_csv(os.path.join(data_dir, 'dev_iam.txt'), sep='\t')
test_df = pd.read_csv(os.path.join(data_dir, 'test_iam.txt'), sep='\t')
all_claims = pd.read_csv(os.path.join(data_dir, 'claims.txt'), sep='\t')
np.random.seed(42)

if AUGMENT_WITH_NEUTRAL_ARGS:
    neutral_claims = all_claims[all_claims.type=='O'] 
    lower_bound = 0
    
    min_train_label = min(train_df['label'].value_counts())
    train_sample = neutral_claims.iloc[:min_train_label]
    train_df = pd.concat([train_df, train_sample]).sample(frac=1)
    lower_bound = min_train_label
    
    min_dev_label = min(dev_df['label'].value_counts())
    dev_sample = neutral_claims.iloc[lower_bound: lower_bound + min_dev_label]    
    dev_df = pd.concat([dev_df, dev_sample]).sample(frac=1)
    lower_bound = lower_bound + min_dev_label
    
    min_test_label = min(test_df['label'].value_counts())
    test_sample = neutral_claims.iloc[lower_bound: lower_bound + min_test_label]    
    test_df = pd.concat([test_df, test_sample]).sample(frac=1)
    
    
label_encoder = LabelEncoder()
label_encoder.fit(train_df['label'])
train_df['label'] = label_encoder.transform(train_df['label'])
dev_df['label'] = label_encoder.transform(dev_df['label'])
test_df['label'] = label_encoder.transform(test_df['label'])

dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'validation': Dataset.from_pandas(dev_df),
    'test': Dataset.from_pandas(test_df)
})

In [76]:
label_encoder.classes_

array([-1,  0,  1])

In [2]:
model_name = 'google/flan-t5-base'

In [3]:
from transformers import AutoTokenizer
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
metric = evaluate.load('rouge')


In [144]:
prefix = 'Given the following topic, generate a good {} argument.'

"""    
need to be careful cause we cant give the same prefix to every type of 
arguments otherwise the model generalises and considers counter arguments as 
supporting arguments if we give the prefix : 'generate a supporting argument'   
"""
def preprocess_function(sample):
    def process_by_type(label_type):
        if label_type==2:
            label = 'supporting'
        elif label_type == 1:
            label = 'neutral'
        else:
            label = 'counter'
            
        label_indices = [i for i, label in enumerate(sample['label']) if label == label_type]        
        prefix = f"Given the following topic, generate a good {label} argument. Topic="
        labeled_samples = {key: [sample[key][i] for i in label_indices] for key in sample.keys()}
        inputs = [prefix + doc for doc in labeled_samples['topic']]

        model_inputs = tokenizer(inputs, max_length=4096, truncation=True)
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(labeled_samples['argument'], max_length=4096, truncation=True)
        model_inputs['labels'] = labels['input_ids']
        return model_inputs
    
    model_inputs_supporting=process_by_type(label_type=2) ## supporting
    model_inputs_neutral=process_by_type(label_type=1) ## neutral
    model_inputs_counter=process_by_type(label_type=0) ## counter
    
    combined_model_inputs = {
    'input_ids': model_inputs_supporting['input_ids'] + model_inputs_neutral['input_ids'] + model_inputs_counter['input_ids'],
    'attention_mask': model_inputs_supporting['attention_mask'] + model_inputs_neutral['attention_mask'] + model_inputs_counter['attention_mask'],
    'labels': model_inputs_supporting['labels'] + model_inputs_neutral['labels'] + model_inputs_counter['labels']
    }
    
    return combined_model_inputs


In [148]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['type', 'id', 'label'])

Map:   0%|          | 0/5644 [00:00<?, ? examples/s]

Map:   0%|          | 0/725 [00:00<?, ? examples/s]

Map:   0%|          | 0/783 [00:00<?, ? examples/s]

In [149]:
batch_size = 8
training_arguments = Seq2SeqTrainingArguments(
    output_dir='results/',
    evaluation_strategy='epoch',
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True, # for cuda
    push_to_hub=False,
    logging_steps=50,
    eval_steps=50,
    save_steps=50,
)


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

In [150]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [151]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: luca-mouchel (lia_epfl). Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disablin

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.402500,3.280517,16.678400,1.980300,13.407000,13.403700,18.600000
2,3.378100,3.253387,16.576600,2.022000,13.417200,13.428700,18.500690
3,3.379300,3.241189,17.009800,1.944100,13.561700,13.580400,18.008276
4,3.338600,3.235717,17.024100,2.083700,13.628100,13.641200,17.957241
5,3.354700,3.233945,16.993000,2.077500,13.642100,13.651900,18.019310


TrainOutput(global_step=1765, training_loss=3.3971486596818012, metrics={'train_runtime': 1254.1659, 'train_samples_per_second': 22.501, 'train_steps_per_second': 1.407, 'total_flos': 1117031829848064.0, 'train_loss': 3.3971486596818012, 'epoch': 5.0})

In [152]:
saved_model = f"models/{model_name.split('/')[-1]}/w_neutral/"
trainer.save_model(saved_model)

In [153]:
predictions = trainer.predict(tokenized_dataset["test"])

In [154]:
predictions.metrics

{'test_loss': 3.1691298484802246,
 'test_rouge1': 16.7889,
 'test_rouge2': 1.6385,
 'test_rougeL': 13.7659,
 'test_rougeLsum': 13.772,
 'test_gen_len': 18.6602809706258,
 'test_runtime': 32.8572,
 'test_samples_per_second': 23.83,
 'test_steps_per_second': 1.491}

In [155]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
model = T5ForConditionalGeneration.from_pretrained(saved_model)
tokenizer = T5Tokenizer.from_pretrained(saved_model)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [180]:
def generate_arg(topic, arg_type='supporting'):
    prefix = f"Given the following topic, generate a good {arg_type} argument. Topic="
    inputs = tokenizer(prefix + topic, return_tensors='pt', padding=True, truncation=True)
    outputs = model.generate(**inputs, min_length=25, max_length=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [202]:
df_iam = pd.read_csv(os.path.join(data_dir, 'train_iam.tsv'), sep='\t')
df_cckg = pd.read_csv(os.path.join(data_dir, 'train_cckg.tsv'), sep='\t')
topic = df_iam['topic'].sample(1).values[0]
print(topic)

Shouldn't alcohol be forbidden


In [203]:
print(topic.upper())

SHOULDN'T ALCOHOL BE FORBIDDEN


In [207]:
arg_types=['supporting', 'counter', 'neutral']

for arg_type in arg_types:
    print(f"""{arg_type}---{generate_arg("Should alcohol be forbidden", arg_type=arg_type)}""")


supporting---Alcohol is a dangerous drug that can cause serious health problems. [ref]. [ref]. Alcohol is a dangerous drug that can cause serious health problems.
counter---Alcohol is a dangerous drug, and it can cause serious health problems. [ref]. [ref]. Alcohol is a dangerous drug, and it can cause serious health problems.
neutral---The alcoholics are able to make a living by drinking alcohol, and they are able to make a living by consuming alcohol.
